In [1]:
import os

import torch
import torch.optim as optim
import torch.nn as nn
from torch.autograd import Variable
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models


os.environ['CUDA_VISIBLE_DEVICES'] = '1'


## Prepare the dataset - CIFAR10

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Files already downloaded and verified
Files already downloaded and verified


## Define a CNN

In [7]:
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()


In [10]:
import resnet


net = resnet.ResNet18()

## Hyperparameters

In [4]:
lr = 1e-4
epochs = 2


## Define the loss function and optimizer

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=lr)


## Train the network

In [15]:
print_steps = 1000
for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.data[0]
        if i % print_steps == print_steps - 1:    # print every print_steps mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / print_steps))
            running_loss = 0.0

print('Finished Training')


[1,  1000] loss: 1.902
[1,  2000] loss: 1.735
[1,  3000] loss: 1.612
[1,  4000] loss: 1.467
[1,  5000] loss: 1.385
[1,  6000] loss: 1.283
[1,  7000] loss: 1.242
[1,  8000] loss: 1.197
[1,  9000] loss: 1.142
[1, 10000] loss: 1.122
[1, 11000] loss: 1.006
[1, 12000] loss: 1.015
[2,  1000] loss: 0.949
[2,  2000] loss: 0.903
[2,  3000] loss: 0.896
[2,  4000] loss: 0.857
[2,  5000] loss: 0.841
[2,  6000] loss: 0.827
[2,  7000] loss: 0.825
[2,  8000] loss: 0.781
[2,  9000] loss: 0.808
[2, 10000] loss: 0.769
[2, 11000] loss: 0.770
[2, 12000] loss: 0.752
Finished Training


## Testing

In [16]:
correct = 0
total = 0
for data in testloader:
    images, labels = data
    outputs = net(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 71 %
